In [1]:
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from tqdm import tqdm
from sklearn.utils import shuffle
#from tensorflow.keras.utils import np_utils
#from tensorflow.keras.layers.convolutional import Convolution2D, MaxPooling2D
#from tensorflow.keras.utils.visualize_util import model_to_dot, plot
from IPython.display import SVG
#import matplotlib.image as mpimg
import matplotlib.image as imread
import cv2
#℅matplotlib inline

In [3]:
#loading the data sets
#train_labels = pd.read_csv("../inputs/planets-dataset/planet/planet/training
train_labels = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
sam_subs = pd.read_csv("../input/planets-dataset/planet/planet/sample_submission.csv")
train_labels.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
#to know the labels that can be assigned to each image
#creating mapping
mapping_csv = train_labels
#summarize property
print(mapping_csv.shape)
print(mapping_csv[:10])

(40479, 2)
  image_name                                         tags
0    train_0                                 haze primary
1    train_1              agriculture clear primary water
2    train_2                                clear primary
3    train_3                                clear primary
4    train_4    agriculture clear habitation primary road
5    train_5                           haze primary water
6    train_6  agriculture clear cultivation primary water
7    train_7                                 haze primary
8    train_8        agriculture clear cultivation primary
9    train_9   agriculture clear cultivation primary road


In [5]:
#creating a set of labels
labels = set()
for i in range(len(mapping_csv)):
    #converting space separated tags into an array of tags
    tags = mapping_csv["tags"][i].split(' ')
    labels.update(tags)

In [6]:
#convert set of labels to a list
labels = list(labels)
#order set alphabetically.
labels.sort()

In [7]:
#making dictionary that maps labels to integers and the reverse
labels_map = {labels[i]:i for i in range(len(labels))}
inv_labels_map = {i:labels[i] for i in range(len(labels))}

In [8]:
#making a function to combine all these
def create_tag_mapping(mapping_csv):
    labels = set()
    for i in range(len(mapping_csv)):
        tags = mapping_csv["tags"][i].split(' ')
        labels.update(tags)
    labels = list(labels)
    labels.sort()
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_labels_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_labels_map
        #labels = list(labels)
        #labels.sort()
        #labels_map = {labels[i]:i for i in range(len(labels))}
        #inv_labels_map = {i:labels[i] for i in range(len(labels))}
        #return labels_map, inv_labels_map
#loading the data
#mapping_csv = pd.read_csv("../inputs/planets-dataset/planet/planet/train_classes.csv")
mapping_csv = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
mapping, inv_mapping = create_tag_mapping(mapping_csv)
print(len(mapping))
print(mapping)
#labels_map, inv_labels_map = create_tag_mapping(mapping_csv)
#print(len(labels_map))
#print(labels_map)
#print(len(inv_labels_map))
    
    

17
{'agriculture': 0, 'artisinal_mine': 1, 'bare_ground': 2, 'blooming': 3, 'blow_down': 4, 'clear': 5, 'cloudy': 6, 'conventional_mine': 7, 'cultivation': 8, 'habitation': 9, 'haze': 10, 'partly_cloudy': 11, 'primary': 12, 'road': 13, 'selective_logging': 14, 'slash_burn': 15, 'water': 16}


In [9]:
#Create a mapping of file name to tags
def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['image_name'][i], mapping_csv["tags"][i]
        mapping[name] = tags.split(' ')
    return mapping    

In [10]:
from os import listdir
from numpy import zeros
from numpy import asarray
from numpy import savez_compressed
from keras.preprocessing.image import img_to_array

#create a mapping of tags yo integers given the loaded mapping file.
def create_tag_mapping(mapping_csv):
    labels = set()
    for i in range(len(mapping_csv)):
        tags = mapping_csv['tags'][i].split(' ')
        labels.update(tags)
    labels = list(labels)
    labels.sort()
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_labels_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_labels_map
#create a mapping of filename to a list of tags
def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv["image_name"][i], mapping_csv["tags"][i]
        mapping[name] = tags.split(' ')
    return mapping
#creating one hot encoding
def one_hot_encode(tags, mapping):
    encoding = zeros(len(mapping), dtype = "uint8")
    for tag in tags:
        encoding[labels_map[tag]]=1
    #for tag in tags.split(' '):
        #encoding[mapping[tag]] = 1
    return encoding
#load all images into memory
def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    for filename in listdir(folder):
        #load image
        photo = load_img(path + filename, target_size =(64,64))
        #convert to numpy areay
        photo = img_to_array(photo,dtype="uint8")
        #get tags
        tag = file_mapping[filename[:-4]]
        #one hot encode tags
        #from sklearn.preprocessing import OneHotEncoder
        #one_hot_encode = OneHotEncoder()
        target = one_hot_encode(tags, tag_mapping)
        photos.append(photo)
        targets.append(target)
    X = asarray(photos, dtype="uint8")
    Y = asarray(targets, dtype="uint8")
    return X,Y
        
        
        
        


In [11]:
#load the mapping file
#filename = train_labels
#filename = train_
filename = "../input/planets-dataset/planet/planet/train_classes.csv"
mapping_csv = pd.read_csv(filename)
#create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(mapping_csv)
#create a maping of filenames to tag list
file_mapping = create_file_mapping(mapping_csv)
#load the jpegs
folder = "../input/planets-dataset/planet/planet/train-jpg/"
#X, Y = load_datasets(folder,file_mapping, tag_mapping)
X, Y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, Y.shape)
savez_compressed("planet_data.npz", X, Y)

(40479, 64, 64, 3) (40479, 17)


#loading the images


In [12]:
#load prepared dataset
#from numpy import load
#data = load("planet_data.npz")
#X, Y = data["arr_0"], data["arr_1"]
print("Loaded: ", X.shape, Y.shape)
#print("Loaded: ', X.shape, Y.shape)
      

Loaded:  (40479, 64, 64, 3) (40479, 17)


In [13]:
#test fbeta score
from numpy import load
from numpy import ones
from numpy import asarray
#from sklearn.preprocessing import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

#load train and test set
def load_dataset():
    data = load("planet_data.npz")
    X, Y = data["arr_0"], data["arr_1"]
    #split into training and test sets
    X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    return X_train, X_test, Y_train, Y_test

#load dataset
X_train, X_test, Y_train, Y_test = load_dataset()
#make all one prediction
train_yhat = asarray([ones(Y_train.shape[1]) for _ in range(Y_train.shape[0])])
test_yhat = asarray([ones(Y_test.shape[1]) for _ in range(Y_test.shape[0])])

#evaluate predictions
#train_score = fbeta_score(Y_train, train_yhat, 2, average="samples")
#train_score = fbeta_score(Y_train, train_yhat, 2, average = "samples")
#test_score = fbeta_score(Y_test, test_yhat, 2, average = "samples")
#print("All ones: train=%.3f, test=%.3f" % (train_score, test_score))

(28335, 64, 64, 3) (12144, 64, 64, 3) (28335, 17) (12144, 17)


In [14]:
from keras import backend as K
#calculating fbeta scoree for multi-class classification
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2
    #just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred,0,1)
    #shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)
    #calculate elements
    tp = K.sum(K.round(y_true * y_pred_bin))+ K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin- y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred_bin, 0, 1)))
    #calculate precision
    p = tp / (tp + fp)
    #calculate recall
    r = tp / (tp + fn)
    #calculate fbeta 
    bb = beta ** 2
    return(bb + 1) * (p * r) / (bb * p + r + K.epsilon())
    
y_true, y_pred = np.round(np.random.rand(100)), np.random.rand(100)


    
    


In [15]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#evaluate predictions with keras
fbeta_keras = fbeta(K.variable(y_true), K.variable(y_pred)).eval(session = K.get_session())
fbeta_sklearn = fbeta_score(y_true, np.round(y_pred), beta=2)
#train_score = fbeta(backend.variable(Y_train), backend.variable(train_yhat))
#test_score = fbeta(backend.variable(Y_test), backend.variable(test_yhat))
print("Scores are {:.3f} (sklearn) and {:.3f} (keras)".format(fbeta_sklearn, fbeta_keras))

2022-09-20 16:11:15.685391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 16:11:15.772810: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 16:11:15.773658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 16:11:15.775941: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Scores are 0.574 (sklearn) and 0.574 (keras)


Evaluating a baseline model****

In [16]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(64, 64, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

In [17]:

model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(17, activation='sigmoid'))

In [18]:
from tensorflow.keras.optimizers import Adam
# compile model
opt = Adam()
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[fbeta])

In [19]:

#create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [20]:
# prepare iterators
train_it = datagen.flow(X_train, Y_train, batch_size=64)
test_it = datagen.flow(X_test, Y_test, batch_size=64)

In [21]:

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# fit model
history = model.fit(train_it, steps_per_epoch=len(train_it),
	validation_data=test_it, validation_steps=len(test_it), epochs=5, verbose=1)

Epoch 1/5


2022-09-20 16:11:20.383612: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


443/443 [==============================] - ETA: 0s - batch: 221.0000 - size: 63.9616 - loss: 0.0024 - fbeta: 0.9990

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


443/443 [==============================] - 14s 15ms/step - batch: 221.0000 - size: 63.9616 - loss: 0.0024 - fbeta: 0.9990 - val_loss: 9.7276e-40 - val_fbeta: 1.0000
Epoch 2/5
443/443 [==============================] - 7s 15ms/step - batch: 221.0000 - size: 63.9616 - loss: 9.9885e-37 - fbeta: 1.0000 - val_loss: 9.7276e-40 - val_fbeta: 1.0000
Epoch 3/5
443/443 [==============================] - 7s 16ms/step - batch: 221.0000 - size: 63.9616 - loss: 9.9883e-37 - fbeta: 1.0000 - val_loss: 9.7276e-40 - val_fbeta: 1.0000
Epoch 4/5
443/443 [==============================] - 7s 15ms/step - batch: 221.0000 - size: 63.9616 - loss: 9.9883e-37 - fbeta: 1.0000 - val_loss: 9.7276e-40 - val_fbeta: 1.0000
Epoch 5/5
443/443 [==============================] - 7s 15ms/step - batch: 221.0000 - size: 63.9616 - loss: 9.9883e-37 - fbeta: 1.0000 - val_loss: 9.7276e-40 - val_fbeta: 1.0000


In [22]:

# evaluate model
loss, fbeta = model.evaluate(test_it, steps=len(test_it), verbose=0)
print('> loss=%.3f, fbeta=%.3f' % (loss, fbeta))

> loss=0.000, fbeta=1.000


In [23]:
#save the model
model.save('/kaggle/working/model_2.h5')


In [24]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_subs[0 : 40669]
files = sam_subs[40669 : ]

In [25]:
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float32)/255.0

100%|██████████| 20522/20522 [02:04<00:00, 165.13it/s]


In [26]:
import gc
gc.collect()

105

In [27]:
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

In [28]:
#converting my encoded labels back to it original form
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = labels_map)

In [29]:
#saving my final result into a csv file
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_subs['tags'] = preds
sam_subs.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [01:55<00:00, 530.74it/s]


In [30]:
sam_subs

,image_name,tags
0,test_0,agriculture cultivation partly_cloudy primary
1,test_1,agriculture cultivation partly_cloudy primary
2,test_2,agriculture cultivation partly_cloudy primary
3,test_3,agriculture cultivation partly_cloudy primary
4,test_4,agriculture cultivation partly_cloudy primary
...,...,...
61186,file_9995,agriculture cultivation partly_cloudy primary
61187,file_9996,agriculture cultivation partly_cloudy primary
61188,file_9997,agriculture cultivation partly_cloudy primary
61189,file_9998,agriculture cultivation partly_cloudy primary
